## Experiment with finding the bit start times, using traces previously collected by otbn_traces.ipynb:

For p384 low-frequency streamed captures.

In [ ]:
import numpy as np
#waves = np.load('waves_p384_22M_half1half0.npy')
#waves = np.load('waves_p384_22M_half1half0_32b_patterned.npy')
waves = np.load('waves_p384_22M_set.npy') # waves[0]: half ones/half zeros; [1]: sames but starts with 0x1fff...; [2]: alternating 32 ones / 32 zeros; [3]: alternating 16 ones / 16 zeros

In [ ]:
base = 0
samples = 600000

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
#datashade(hv.Curve(waves[0][base:base+samples]-waves[9][base:base+samples])).opts(width=2000, height=900)
datashade(hv.Curve(waves[0][base:base+samples])).opts(width=2000, height=800)

By looking at the difference between `waves[0]` (k=0xfff...) and `waves[1]` (k=0x1fff...) we can observe the leading 1 leakage and at the same time confirm when the first bit of k is processed:

In [ ]:
datashade(hv.Curve(waves[0][base:base+samples] - waves[1][base:base+samples])).opts(width=2000, height=800)

Next we apply a high-pass filter to clean up the trace. We want to flatten out the idle trace portions that separate the bits, so that we can more easily establish bit markers.

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=9):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
filtered_waves = []
for wave in waves:
    filtered_waves.append(butter_highpass_filter(wave, 1e6, 100e6))

Visualize the transformation:

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

w0 = datashade(hv.Curve(waves[0][base:base+samples]), cmap=['green'])
wf = datashade(hv.Curve((filtered_waves[0][base:base+samples])), cmap=['black'])

(w0 * wf).opts(width=2000, height=900)
#(wf).opts(width=2000, height=600)

Now we're ready to use `waves[0]` to try to guess the bit start times.

We'll be using the idle periods as a marker. Since the difference between idle / not idle is not so clean, a simple threshold may not work so well, so let's apply a moving average:

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
mfw = moving_average(np.abs(filtered_waves[0]), 1500)

In [ ]:
mwf = datashade(hv.Curve(mfw[base:base+samples]), cmap=['black'])
mwf.opts(width=2000, height=600)

Looks nice and clean now! Next we pick out the peaks:

In [ ]:
from scipy.signal import find_peaks
peaks, _ = find_peaks(-mfw[base:base+samples], distance=20000)

In [ ]:
len(peaks), peaks

Note that we're not processing through the full power trace -- you'll see why soon.

From the first plots at the start of this notebook, we're guessing that the first bit processing starts around sample 70000. That would be the fourth peak in the list above.

We are also guessing that there are two peaks per bit. So we assemble our initial guess at bit start times like this:

In [ ]:
bit_starts = peaks[3::2]

In [ ]:
len(bit_starts)

As a sanity check let's see where the chosen times are located:

In [ ]:
from operator import mul
from functools import reduce

starts = [hv.VLine(b) for b in bit_starts]
(mwf * reduce(mul, starts)).opts(width=2000, height=600)

Now let's look at the time delta between each bit:

In [ ]:
deltas = []
for i in range(len(bit_starts)-2):
    delta = bit_starts[i+1] - bit_starts[i]
    deltas.append(delta)
    print(delta)
    

In [ ]:
avg_duration = int(np.average(deltas))
avg_duration, max(deltas)-min(deltas)

The deltas are pretty consistent but there is some variation.

As a sanity check, this should be a number less than the total operation time reported during trace acquisition:

In [ ]:
avg_duration*384

Now we can plot the power trace of each bit and see how they align:

In [ ]:
bits = []
for start in bit_starts:
    bits.append(filtered_waves[0][start:start+avg_duration])

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

from operator import mul
from functools import reduce

#curves = [hv.Curve(zip(xrange, filtered_waves[0][bit_starts[i]:bit_starts[i]+duration])) for i in range(numbits)]
curves = [hv.Curve(bits[i]) for i in range(len(bits))]

reduce(mul, curves).opts(width=2000, height=900)

Zoom in and you'll find not-so-great alignment.

Let's guess that the bit processing times are actually constant. We adjust our guess for the bit start times like this:

In [ ]:
even_starts = []
duration = avg_duration
#duration = 46610
for i in range(256):
    even_starts.append(bit_starts[0] + i*duration)

In [ ]:
bits = []
for start in even_starts:
    bits.append(filtered_waves[0][start:start+avg_duration])

In [ ]:
numbits = 10
#numbits = len(bits) # possible but slow!
curves = [hv.Curve(bits[i]) for i in range(numbits)]
reduce(mul, curves).opts(width=2000, height=900)

Some tweaking to `avg_duration` may be required, but with the correct value you should see perfectly aligned peaks every 10 samples throughout.

# Difference of Means attack

Let's see if simple difference-of-means reveals anything.

Using `waves[0]` we'll compute the average power trace for k=1 and for k=0, and see whether the difference between these two averages reveals a simple distinguisher that could be used to guess arbitrary k.

In [ ]:
avg_trace = np.zeros(duration)
avg_ones = np.zeros(duration)
avg_zeros = np.zeros(duration)

for i in range(len(bit_starts)):
    avg_trace += bits[i][:duration]
    if i < 128:
        avg_ones += bits[i][:duration]
    else:
        avg_zeros += bits[i][:duration]

avg_trace /= len(bit_starts)
avg_ones /= len(bit_starts)/2
avg_zeros /= len(bit_starts)/2


In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

cavg_all = datashade(hv.Curve(avg_trace), cmap=['black'])
cavg_ones = datashade(hv.Curve(avg_ones), cmap=['blue'])
cavg_zeros = datashade(hv.Curve(avg_zeros), cmap=['green'])

cdiff = datashade(hv.Curve((avg_ones - avg_zeros)*10), cmap=['red'])

#(cavg_all * cavg_ones * cavg_zeros).opts(width=2000, height=900)
#(cavg_zeros * cavg_ones).opts(width=2000, height=900)
#(cavg_ones * cavg_zeros).opts(width=2000, height=900)
(cdiff).opts(width=2000, height=600)

There does seem to be a handful of sample times with a clear difference:

In [ ]:
markers = np.where((avg_ones - avg_zeros) > 0.003)[0]

In [ ]:
len(markers) #, markers

In [ ]:
# save markers for attacking other traces:
#np.save('markers_p384_25M.npy', markers)

In [ ]:
#markers = np.load('markers_25M.npy')

In [ ]:
scores = []
for b in range(len(bit_starts)):
    score = 0
    for marker in markers:
        score += bits[b][marker]
    scores.append(score)

In [ ]:
cscores = hv.Curve(scores)
(cscores * hv.VLine(128)).opts(width=2000, height=600)

This does seem to work: while we couldn't successfully guess all bits of k from this, there is a clear difference between the first and second halves.

(Remember, k = {128 ones, 256 zeros} for this. We're only looking at the first 256 bits of k out of convenience because the full operation wasn't captured.)

Before declaring victory, let's see what happens if k is less regular. Let's start with `waves[2]`, which has `k=0xffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000ffff0000`.

We'll keep the same markers but feed in a different traces. We will assume that the bit start times are unchanged (during trace acquisition we saw that the operation time did not change so this should be a safe bet).

In [ ]:
bits = []
for start in even_starts:
    bits.append(filtered_waves[2][start:start+avg_duration])

In [ ]:
scores = []
for b in range(len(bit_starts)):
    score = 0
    for marker in markers:
        score += bits[b][marker]
    scores.append(score)

In [ ]:
cscores = hv.Curve(scores)
lines = [hv.VLine(b) for b in range(0, 256, 16)]
(cscores * reduce(mul, lines)).opts(width=2000, height=600)

Perhaps a little less clear visually, but numerically there is a difference:

In [ ]:
ones = 0
zeros = 0
for b in range(256):
    if b % 32 < 16:
        ones += scores[b]
    else:
        zeros += scores[b]

In [ ]:
ones, zeros

And finally, `waves[3]`, which has `k=0xff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00ff00`:

In [ ]:
bits = []
for start in even_starts:
    bits.append(filtered_waves[3][start:start+avg_duration])

In [ ]:
scores = []
for b in range(len(bit_starts)):
    score = 0
    for marker in markers:
        score += bits[b][marker]
    scores.append(score)

In [ ]:
cscores = hv.Curve(scores)
lines = [hv.VLine(b) for b in range(0, 256, 8)]
(cscores * reduce(mul, lines)).opts(width=2000, height=600)

In [ ]:
ones = 0
zeros = 0
for b in range(256):
    if b % 16 < 8:
        ones += scores[b]
    else:
        zeros += scores[b]

ones, zeros